# Исследование надежности заемщиков

**Цель исследования** — проверить 4 гипотезы, чтобы принять решение об учете дополнительных фактороа в модели кредитного скоринга:
1. Наличие детей влияет на факт погашения кредита в срок, т.е. если есть хотя бы один ребенок, то кредит будет погашен в срок с большей вероятностью.
2. Семейное положение влияет на факт погашения кредита в срок, т.е. если заемщик находится в браке или граждонском браке, то кредит будет погашен в срок с большей вероятностью.
3. Чем выше уровень дохода, тем вероятнее возврат кредита в срок
4. Разные цели кредита влияют на его возврат в срок

**Ход исследования**
Данные о поведении пользователей получены из файла. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор и исправление данных. 

**Исследование пройдёт в три этапа**:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('data_2.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В каждой строке таблицы — данные о заемщике, целе заема и наличия задолженности:
    
* childreт — количество детей
* days_employed — трудовой стаж в месяцах 
* dob_years — возраст заемщика скорее всего
* education — образование
* education_id — идентификатор образования
* family_status — семейное положение ['женат / замужем', 'гражданский брак', 'вдовец / вдова', 'в разводе', 'Не женат / не замужем']
* family_status_id — идентификатор семейного положения
* gender —  пол
* income_type — статус, влияющий на способ прихода денег ['сотрудник', 'пенсионер', 'компаньон', 'госслужащий', 'безработный', 'предприниматель', 'студент', 'в декрете']
* debt — есть ли еще заемы: 0-нет, 1- да(?)
* total_income — доход, получаемый в месяц в рублях (?)
* purpose — цель заема
     

In [2]:
data.info()
#data['income_type'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Тип данных разный.
В названиях колонок нет пробем (все на англ, змеиный регистр, строчные буквы, нет пробелов)
Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

### Шаг 2.1 Заполнение пропусков

Посмотрим сколько и где в таблице пропущенных значений

In [3]:
cell_with_null = data.isna().sum()
cell_with_null

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Поймем какую долю составляют пропущенные значения в каждом из колонок, где есть пропуски: 'days_employed' и 'total_income'. 

In [4]:
total_row = 21525 #взято шага 1
    
print('days_employed', data['days_employed'].isna().sum()/total_row)
print('total_income ', data['total_income'].isna().sum()/total_row)

days_employed 0.10099883855981417
total_income  0.10099883855981417


Так как количество и доля пропущенных значений одинаковая, то можно предположить, что пропуски в обоих колонках в одинаковых строчках. Например, если заемщик ни разу не работал, то у него и не будет дохода, например, у студентов или в декрете 

In [5]:
data_test = data[data['days_employed'].isna()]
data_test.groupby('income_type')['total_income'].sum()

income_type
госслужащий        0.0
компаньон          0.0
пенсионер          0.0
предприниматель    0.0
сотрудник          0.0
Name: total_income, dtype: float64

Из полученных данных видно, что если кол-во дней стажа пустое, то и доход заемщика тоже пустой.
При этом такая ситуация наблюдается не для студентов, которые могли еще не работать.
Получается пустые данные - это результат ошибки ввода.

Доля пропущеных значений не маленькая - 10%, и удалять такие стоки не стоит, чтобы не уменишать полезную выборку
Так как пропуски обнарушены в колонках с количественными переменными, то имеет смысл их заполнить значениями, характеризующими состояния выборки. Подойдет медианное значение, а не среднее, чтобы не было влияния на сильно выделющиеся значения выборки.

Перед тем как рассчитывать медины и запонять ими пустые значения проверим остальные данные в этих колонках на адекватность

In [6]:
# data[data['days_employed'] < 0]
data.loc[data['days_employed'] < 0, 'days_employed' ].count()

15906

In [7]:
# data[data['total_income'] < 0]
data.loc[data['total_income'] < 0, 'total_income' ].count()

0

Заполнить пустые значения мы можем только для дохода 'total_income', так как там нет отрицательных значений
Для трудового стажа 'days_employed' сделаем это после исправления данных.

In [8]:
total_income_median = data['total_income'].median()

# Заполняем пропуски медианой для доходв
data['total_income'] = data['total_income'].fillna(total_income_median)
#data.loc[data['total_income'].isna(),'total_income'] = total_income_median  - второй вариант

In [9]:
# Проверим себя
data.isna().sum()

# Дригие варианты
#data[data['total_income'].isna()]['total_income']
# data.loc[data['total_income'] < 0, 'total_income' ].count()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

**Разберемся аномалиями в колонке "days_employed"**

Сначала узнаем у каких заемщиков отрицательный стаж, а у каких положительный, потому что кол-во дней работы зависит от таго чем человек занимается и косвенно помно понять сколько он уже проработал.

In [10]:
test = data[data['days_employed'] < 0]
test.groupby('income_type')['income_type'].count().sort_values()

income_type
в декрете              1
предприниматель        1
студент                1
госслужащий         1312
компаньон           4577
сотрудник          10014
Name: income_type, dtype: int64

In [11]:
test = data[data['days_employed'] > 0]
test.groupby('income_type')['income_type'].count().sort_values()

income_type
безработный       2
пенсионер      3443
Name: income_type, dtype: int64

Всего 8 типов заемщиков. Для 6-ти типов заемщиков указывался стаж как отрицательный, для остальных 2-х типов - положительный.
Похоже на ошибку ввода данных, по этому отрацательные значения взьмем по модулю, чтобы привести к единому виду

In [12]:
# Применем модуль к отрицательным данным
data['days_employed'] = data['days_employed'].abs()

In [13]:
#Проверим себя
data.loc[data['days_employed'] < 0, 'days_employed' ].count()

0

In [14]:
print('Возраст заемщика с максимальным стажем', data.loc[data['days_employed'] == data['days_employed'].max(), 'dob_years'])
print('Стаж заемщика с максимальным возрастом', data.loc[data['dob_years'] == data['dob_years'].max(), 'days_employed'])

Возраст заемщика с максимальным стажем 6954    56
Name: dob_years, dtype: int64
Стаж заемщика с максимальным возрастом 8880    1678.969771
Name: days_employed, dtype: float64


Выведенные результат пока ничего не говорят, так как не обязательно большой стаж должен быть у самого взрослого займщика.

Начнем с того, что попытаемся понять в каких единицах измерения стаж в таблице - дни или часы.

Для этого посчитаем максимальный стаж для заемщика с максимальным возрастом из данных - 75 лет и, допустим, он работал с 16 лет, тогда получается что его стаж должен быть 59 лет. 
Если среднее кол-во рабочих дней в году 247, то максимальный стаж должен быть:
14573 - дней 
или 
116584 - часов

Проверим кол-во записей удовлеворяющим данным условиям и их долю

In [15]:
a = data.loc[data['days_employed'] > 14573, 'dob_years'].count()
print('Кол-во заемщиков, где стаж больше 14573 дней -', a)
print('Доля заемщиков с аномалиями в стаже -', a / total_row)

Кол-во заемщиков, где стаж больше 14573 дней - 3463
Доля заемщиков с аномалиями в стаже - 0.16088269454123114


In [16]:
a = data.loc[data['days_employed'] > 116584, 'dob_years'].count()
print('Кол-во заемщиков, где стаж больше 116584 часов -', a)
print('Доля заемщиков с аномалиями в стаже -', a / total_row)

Кол-во заемщиков, где стаж больше 116584 часов - 3445
Доля заемщиков с аномалиями в стаже - 0.16004645760743322


Итак:
1. Так как кол-во записей, где стаж превышает стаж в часах и в минутах практически одинаковое, то можно сделать вывод, что стаж в таблице в днях и это ошибочные данные
2. Доля заемщиков со слишком большим стажем около 16%, но этому такие строки стоит исправить. Предлагаю указать там максимальный стаж в днях

In [17]:
# Заполним большие значения максимальным в днях
max_days_employed = 14573
data.loc[data['days_employed'] > max_days_employed, 'days_employed'] = max_days_employed

In [18]:
#Проверим себя

data['days_employed'].max()
#data.loc[data['days_employed'] >= 14573, 'dob_years'].count()

14573.0

А теперь можно посчитать медиану и заполнить пропуски:

In [19]:
days_employed_median = data['days_employed'].median()
days_employed_median

2194.220566878695

In [20]:
# Заполним пустые значения медианным
data['days_employed'] = data['days_employed'].fillna(days_employed_median)

In [21]:
# Проверим, что пропусков нет
data.isna().sum()  


#data[data['days_employed'].isna(), 'days_employed']

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Размеремся аномалиями в колонке "children"**

Проверим какие значения принимает колонка с количестовм детей

In [22]:
data['children'].value_counts().sort_values()

 5         9
 4        41
-1        47
 20       76
 3       330
 2      2055
 1      4818
 0     14149
Name: children, dtype: int64

*Нужно исправить аноманию детей равное '-1'*
1. Возьмем заемщиков, у которых есть дети и проверим распределение их family_status (можно, конечно, еще и по доходу посмотреть, но симейное положение важнее)
2. Проверим, как распределился family_status для тех, у котого есть дети и тех, у кото отрицательное кол-во детей
3. Если распеределение похожее, то принимем решение, что знак "минус" становлен по ошибке 

In [23]:
test = data[data['children'] < 0]
test.groupby('family_status')['family_status'].count()

family_status
Не женат / не замужем     5
в разводе                 4
вдовец / вдова            4
гражданский брак          5
женат / замужем          29
Name: family_status, dtype: int64

In [24]:
test = data[data['children'] > 0]
test.groupby('family_status')['family_status'].count()

family_status
Не женат / не замужем     543
в разводе                 407
вдовец / вдова            108
гражданский брак         1420
женат / замужем          4851
Name: family_status, dtype: int64

Распределение в обоих случаях похожее, соответвенно, можно сделать вывод, что отрицательным знаком не помечали отсутствие детей, а просто ошиблись.
По этому заменим -1 на 1

In [25]:
data.loc[data['children'] == -1, 'children'] = 1
#Проверим себя
data['children'].value_counts().sort_values()

5         9
4        41
20       76
3       330
2      2055
1      4865
0     14149
Name: children, dtype: int64

Посмотрев на распределение кол-ва детей, можно сделать вывод, что есть большой выброс, когда детей 20. Это явная аномалия, которую нужно исправить.
Есть два варианта:
1. Это ошибка ввода и должно быть не дватцать, а  два ребенка
2. 20 - так отмечали детей, когда знали, что их достаточно много, но не хотели указывать сколько 

Если не считать выброса (20 детей), распределение показывает, что максимально 5 детей было только у 9 человек и, соответвенно, если следовать распределению, то следующее значение по кол-ву детей должно быть у менее 9 заемщиков, а у нас для 20 детей - 76 заемщиков. По этому своем решении я предлагаю использовать 1 вариант и исправить 20 на 2

In [26]:
data.loc[data['children'] == 20, 'children'] = 2
#Проверим себя
data['children'].value_counts().sort_values()

5        9
4       41
3      330
2     2131
1     4865
0    14149
Name: children, dtype: int64

**Разберемся аномалиями в колонке "dob_years "**
* Проверим, что заемщику больше 18 лет
* Проверим максимальный возраст заемщика

In [27]:
print('Какой возраст заемщиков и сколько таких строк ', data.loc[data['dob_years'] < 18, 'dob_years'].value_counts())
print('Доля заемщиков меньше 18 лет = ', data[data['dob_years'] < 18]['dob_years'].count() / total_row)

print('Семейный статус заемщиков меньше 18 лет', data.loc[data['dob_years'] < 18, 'family_status'].unique())

Какой возраст заемщиков и сколько таких строк  0    101
Name: dob_years, dtype: int64
Доля заемщиков меньше 18 лет =  0.004692218350754936
Семейный статус заемщиков меньше 18 лет ['женат / замужем' 'в разводе' 'Не женат / не замужем' 'вдовец / вдова'
 'гражданский брак']


Из полученной информации можно сделать вывод, что возраст меньше 18 лет - это ошибка.
Вижу 2 варианта как исправть:
1. Так как доля заемщиков меньше 18 лет меньше процента, то такие строки можно исключить
2. Такие строки можно оставить, заполнив среднем значением, так как в рамках проверяемых гипотиз возраст имеет не самое важное значения 

Выбираю второй вариает, предварительно только проверю адекватное ли максимальное значение

In [28]:
print(data['dob_years'].max())

75


Посчитаем среднее и заполним из возраст, равный нулю:

In [29]:
mean_years = data.loc[data['dob_years'] > 18, 'dob_years'].mean()
data.loc[data['dob_years'] == 0,'dob_years'] = mean_years

In [30]:
# Проверим, что пропусков нет
data[data['dob_years'] == 0]['dob_years']
#data['dob_years'].value_counts()

Series([], Name: dob_years, dtype: float64)

### Шаг 2.3. Изменение типов данных
Заменим вещественный тип данных в столбце total_income на целочисленный

In [31]:
data['total_income'] = data['total_income'].astype('int')

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(2), int32(1), int64(4), object(5)
memory usage: 1.9+ MB


### Шаг 2.4. Удаление дубликатов
Проверим явные и не явные дубликаты

In [33]:
data.duplicated().sum()

54

In [34]:
# Удалим явные дубликаты
data = data.drop_duplicates().reset_index(drop=True)

In [35]:
# Убедимся, что полностью избавились от явных дубликатов
data.duplicated().sum()

0

In [36]:
# Поищем неявные дубликаты в колонке "purpose"
purpose_test = data['purpose'].sort_values()
purpose_test.unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

In [37]:
# Функция для замены неявных дубликатов

def replace_wrong_genres(wrong, correct):
    for wrong_value in wrong: 
          data['purpose']= data['purpose'].replace(wrong_value, correct)

In [38]:
# Разберемся с автомобилями
wrong = ['автомобили', 'на покупку автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'приобретение автомобиля', 'свой автомобиль','сделка с автомобилем', 'сделка с подержанным автомобилем']
correct = 'автомобиль'

replace_wrong_genres(wrong, correct)

In [39]:
# Разберемся с образованием
wrong = [ 'высшее образование', 'дополнительное образование', 'заняться высшим образованием', 'заняться образованием', 'получение высшего образования', 'получение дополнительного образования', 'получение образования', 'профильное образование']
correct = 'образование'

replace_wrong_genres(wrong, correct)

In [40]:
# Разберемся с недвижимостью
wrong = ['операции с жильем', 'жилье', 'операции с коммерческой недвижимостью','операции с недвижимостью', 'операции со своей недвижимостью','покупка жилой недвижимости', 'покупка жилья','покупка жилья для сдачи', 'покупка жилья для семьи','покупка коммерческой недвижимости', 'покупка недвижимости','покупка своего жилья']
correct = 'недвижимость'

replace_wrong_genres(wrong, correct)

In [41]:
# Разберемся со свадьбой
wrong = ['на проведение свадьбы', 'сыграть свадьбу']
correct = 'свадьба'

replace_wrong_genres(wrong, correct)

In [42]:
# Разберемся со строительством
wrong = ['строительство жилой недвижимости', 'строительство недвижимости','строительство собственной недвижимости']
correct = 'строительство'

replace_wrong_genres(wrong, correct)

In [43]:
# Разберемся со ремонтом
wrong = ['ремонт жилью']
correct = 'ремонт'

replace_wrong_genres(wrong, correct)

In [44]:
# Проверим дубликаты в колонке "purpose"
purpose_test = data['purpose'].sort_values()
purpose_test.unique()

array(['автомобиль', 'недвижимость', 'образование', 'ремонт', 'свадьба',
       'строительство'], dtype=object)

Теперь цели привидены в порядок и четко классифицированы

In [45]:
# Поищем неявные дубликаты в колонке 'education'
education_test = data['education'].sort_values()
education_test.unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

В данном случае нужно привести все образование к нижнему регистру

In [46]:
wrong_values_count = 0

for i in range(total_row): 
    try:
        data.loc[i,'education'] = data.loc[i,'education'].lower()   
    except:
        wrong_values_count += 1

wrong_values_count 

# Потом нашла, что это можно сделать проще для всего столбца:
# df=data['education']
# df.str.lower()

54

In [47]:
#Проверим смену регистра
education_test = data['education'].sort_values()
education_test.unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

Остальные качественные колонки проверила и там нет проблем, не буду тут эти строки выводить

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:
* каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
* каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

Чтобы:
* Не увеличивался размер файла и время обработки данных
* Не ошибаться при фильтрации данных по типу обращения
* создание новых категорий и изменение старых не отнимло много времени

In [48]:
# выделим столбцы для справочника
education_dict = data[['education', 'education_id']]
family_dict = data[['family_status', 'family_status_id']]

In [49]:
# удалим дубликаты в справочнике
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [50]:
# удалим дубликаты в справочнике
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
family_dict.head

<bound method NDFrame.head of            family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4>

In [51]:
# Оставим в исходном датафрейме только id
data = data.drop(['family_status', 'education'], axis=1)
data

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,0,0,F,сотрудник,0,253875,недвижимость
1,1,4024.803754,36.0,1,0,F,сотрудник,0,112080,автомобиль
2,0,5623.422610,33.0,1,0,M,сотрудник,0,145885,недвижимость
3,3,4124.747207,32.0,1,0,M,сотрудник,0,267628,образование
4,0,14573.000000,53.0,1,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43.0,1,1,F,компаньон,0,224791,недвижимость
21467,0,14573.000000,67.0,1,0,F,пенсионер,0,155999,автомобиль
21468,1,2113.346888,38.0,1,1,M,сотрудник,1,89672,недвижимость
21469,3,3112.481705,38.0,1,0,M,сотрудник,1,244093,автомобиль


### Шаг 2.6. Категоризация дохода

На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [52]:
#  Функция для определения категории
def income_category(total_income):
    try:
        if total_income <= 30000:
            return 'E'
        elif (total_income >= 30001 and total_income <= 50000):
            return 'D'
        elif (total_income >= 50001 and total_income <= 200000):
            return 'C'  
        elif (total_income >= 200001 and total_income <= 1000000):
            return 'B' 
        elif total_income >= 1000001:
            return 'A' 
    except:
        'категория не определена'
        

In [53]:
# Определим категорию для кажой ячейки
data['total_income_category'] = data['total_income'].apply(income_category)
data.head(3)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42.0,0,0,F,сотрудник,0,253875,недвижимость,B
1,1,4024.803754,36.0,1,0,F,сотрудник,0,112080,автомобиль,C
2,0,5623.422610,33.0,1,0,M,сотрудник,0,145885,недвижимость,C


In [54]:
# Проверим категории
data.value_counts('total_income_category')

total_income_category
C    16033
B     5041
D      350
A       25
E       22
dtype: int64

### Шаг 2.7. Категоризация целей кредита
Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'

Практически полностью я эту задачу решила в этапе 2.4 Удаление дубликатов.
Я создала следующие цели - 'автомобиль', 'недвижимость', 'образование', 'ремонт', 'свадьба', 'строительство'
В рамках данной задачи предлагаю цели 'ремонт' и 'строительство' перенести в новую категорию -> 'операции с недвижимостью'

In [55]:
#  Функция для категориизации целей
def purpose_category(row):
    purpose = row['purpose']
    try:
        if purpose == 'автомобиль':
            return 'операции с автомобилем'
        elif (purpose == 'недвижимость' or purpose == 'ремонт' or purpose == 'строительство'):
            return 'операции с недвижимостью'
        elif purpose == 'образование':
            return 'получение образования'    
        elif purpose == 'свадьба':
            return 'проведение свадьбы' 
    except:
        return 'не определено'

In [56]:
#  Установим категории и проверим что получилось
data['purpose_category'] = data.apply(purpose_category, axis=1)
data.head(3)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42.0,0,0,F,сотрудник,0,253875,недвижимость,B,операции с недвижимостью
1,1,4024.803754,36.0,1,0,F,сотрудник,0,112080,автомобиль,C,операции с автомобилем
2,0,5623.422610,33.0,1,0,M,сотрудник,0,145885,недвижимость,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [57]:
# Для семей с разным кол-вом детей опредеим долю наличия заполжностей
for i in range(6):
        print('Детей - ', i, ', Доля задолжностей - ', data[(data['children'] == i) & (data['debt'] == 1)]['debt'].count() / data[data['children'] == i]['debt'].count())        

Детей -  0 , Доля задолжностей -  0.07535266179910682
Детей -  1 , Доля задолжностей -  0.09163920922570017
Детей -  2 , Доля задолжностей -  0.09492481203007519
Детей -  3 , Доля задолжностей -  0.08181818181818182
Детей -  4 , Доля задолжностей -  0.0975609756097561
Детей -  5 , Доля задолжностей -  0.0


In [58]:
print ('Средняя доля задолжностей для семей с детьми: ', (0.09163920922570017 + 0.09492481203007519 + 0.08181818181818182 + 0.0975609756097561) / 4)
print ('Отклонение доли задолжностей самай с детьми и без : ', 0.09148579467092832 - 0.07535266179910682)

Средняя доля задолжностей для семей с детьми:  0.09148579467092832
Отклонение доли задолжностей самай с детьми и без :  0.0161331328718215


Из данных видно, что на возврат заема в срок влияет не кол-во детей, скорее влияет факт наличия хотя бы одного ребенк, но это влияние не значительное.
Для семей с 5 детьми не найдено ни одной задолжности, это скорее всего связано с тем, что таких семей мало относительно других и выборка получилась не показательная.
Общий вывод, что семьи с детьми чаше на возвращают заемы (9%), чем без детей

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [59]:
# Для каждого семейного положения считаем кол-во кредиов и ко-во не возвратов
family_status_grouped = data.groupby('family_status_id').agg({'debt': ['count', 'sum']})
family_status_grouped

debt     
                  count  sum
family_status_id            
0                 12344  931
1                  4163  388
2                   959   63
3                  1195   85
4                  2810  274

In [60]:
# Для каждого семейного положения считаем % не возвратов
family_status_grouped['dole'] = family_status_grouped['debt']['sum'] / family_status_grouped['debt']['count']
family_status_grouped

debt           dole
                  count  sum          
family_status_id                      
0                 12344  931  0.075421
1                  4163  388  0.093202
2                   959   63  0.065693
3                  1195   85  0.071130
4                  2810  274  0.097509

In [61]:
# Добавляем тектовый семейный статус из справочника
family_status_grouped.merge(family_dict, on='family_status_id', how='left')

C:\Users\e.chernykh\Anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


,family_status_id,"(debt, count)","(debt, sum)","(dole, )",family_status
0,0,12344,931,0.075421,женат / замужем
1,1,4163,388,0.093202,гражданский брак
2,2,959,63,0.065693,вдовец / вдова
3,3,1195,85,0.071130,в разводе
4,4,2810,274,0.097509,Не женат / не замужем


Вывод сделать сложно, возможно, есть еще какие-то дополнительные факторы (возможно дети?), которые влияют на результат. Тут нужно дополниельное исследование.

Но явно видно, что не жетатые и не замужние и, в целом, более свободные люди более склонны не возвращать заемы.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [62]:
# Создадим сводную таблицу
data_pivot = data.pivot_table(index='total_income_category', columns = 'debt', values='purpose', aggfunc='count')
data_pivot

debt,0,1
total_income_category,,
A,23,2
B,4685,356
C,14673,1360
D,329,21
E,20,2


In [63]:
# Посчитаем долю невозвратов
data_pivot['dole'] = data_pivot[1] / (data_pivot[1] + data_pivot[0])
data_pivot

debt,0,1,dole
total_income_category,,,
A,23,2,0.080000
B,4685,356,0.070621
C,14673,1360,0.084825
D,329,21,0.060000
E,20,2,0.090909


Когда посчитали долю невозвратов, оказалось, что % невозвратов у всех категорий доходов приблизительно одинаков и отличается на 3%. Из них максимальный - 9% у категории E, где самя низкая сумма заема, а минимальный у  категории D (30001–50000)

##### Вопрос 3: Как разные цели кредита влияют на его возврат в срок?

In [64]:
# Создадим сводную таблицу
data_pivot = data.pivot_table(index='purpose_category', columns = 'debt', values='purpose', aggfunc='count')
data_pivot

debt,0,1
purpose_category,,
операции с автомобилем,3905,403
операции с недвижимостью,10032,782
получение образования,3644,370
проведение свадьбы,2149,186


In [65]:
# Посчитаем долю невозвратов
data_pivot['dole'] = data_pivot[1] / (data_pivot[1] + data_pivot[0])
data_pivot

debt,0,1,dole
purpose_category,,,
операции с автомобилем,3905,403,0.093547
операции с недвижимостью,10032,782,0.072314
получение образования,3644,370,0.092177
проведение свадьбы,2149,186,0.079657


Для операций с недвижимостью кол-во невозвратом минимальное, возможно это связно с дополнительными условиями стахования. Заемы на проведение свадьбы тоже хорошо возвращаются относительно друх категорий.
Более 9% невозвратов у оперций с автомобилями и образованием, возможно свзано с авариями и изменением планов у студентов.

## Общий вывод:

1. Семьи с детьми чаше на возвращают заемы (9%), чем без детей
2. Не состоящие в браке люди склонны не возвращать заемы, таких практически 10%
3. Максимальная вероятность невозврата - 9% у заемов на маленькие суммы(менее 30.000р)
4. Более 9% невозвратов у оперций с автомобилями и образованием, возможно свзано с авариями и изменением планов у студентов.

В целом процент невозратов в районе 9%, максимальный - у не состоящих в браке людей. Предлагаю добавить в скоритнговую модель в первую очередь этот фактор.
